In [1]:
%matplotlib inline
from collections import Counter
from typing import Callable, List, Dict
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql import functions as F
import pyspark as ps
import json
import requests
import scipy.stats as stats

# Always make it pretty.
plt.style.use('ggplot')

/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# Create spark session
spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('julia_json')
         .getOrCreate()
        )
sc = spark.sparkContext

In [3]:
spark_df = spark.read.json('data/french_tweets.json')
spark_df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [4]:
spark_df.show(5)

+------------+-----------+--------------------+------------------+--------------------+-----------------+--------------+--------------+---------+------------+----+------------------+------------------+-----------------------+---------------------+-------------------------+-------------------+-----------------------+---------------+----+-----+--------------------+------------------+-------------+----------------+--------------------+-------------+---------+--------------------+--------------------+-------------+---------+--------------------+
|contributors|coordinates|          created_at|display_text_range|            entities|extended_entities|extended_tweet|favorite_count|favorited|filter_level| geo|                id|            id_str|in_reply_to_screen_name|in_reply_to_status_id|in_reply_to_status_id_str|in_reply_to_user_id|in_reply_to_user_id_str|is_quote_status|lang|limit|               place|possibly_sensitive|quoted_status|quoted_status_id|quoted_status_id_str|retweet_count|re

In [43]:
macron_tweets = spark_df.select('text', 'retweet_count').filter(spark_df.text.like('%macron%'))

In [49]:
rdd = macron_tweets.rdd
rdd1 = rdd.flatMap(lambda x: x[0])
# rdd2 = rdd1.map(lambda x: x.split('')
rdd1.take(100)

['L',
 'a',
 ' ',
 'm',
 'a',
 'c',
 'r',
 'o',
 'n',
 'i',
 'e',
 ' ',
 'E',
 'n',
 ' ',
 'M',
 'a',
 'r',
 'c',
 'h',
 'e',
 ' ',
 '!',
 ' ',
 'c',
 'o',
 'm',
 'm',
 'e',
 'n',
 'c',
 'e',
 ' ',
 'd',
 'é',
 'j',
 'à',
 ' ',
 'à',
 ' ',
 's',
 "'",
 'é',
 'c',
 'h',
 'a',
 'r',
 'p',
 'e',
 'r',
 ' ',
 'a',
 'v',
 'a',
 'n',
 't',
 ' ',
 'd',
 "'",
 'a',
 'v',
 'o',
 'i',
 'r',
 ' ',
 'c',
 'o',
 'm',
 'm',
 'e',
 'n',
 'c',
 'é',
 ' ',
 ':',
 '\n',
 '\n',
 'h',
 't',
 't',
 'p',
 's',
 ':',
 '/',
 '/',
 't',
 '.',
 'c',
 'o',
 '/',
 '9',
 'f',
 '4',
 'Y',
 'h',
 'j',
 'M',
 'F',
 'a',
 'G']

In [59]:
rdd1.count()

101460

In [58]:
x = spark.createDataFrame(rdd1)
x2 = x.keys()
x2.take(10)


TypeError: Can not infer schema for type: <class 'str'>

In [51]:
macron_tweets.show()

+--------------------+-------------+
|                text|retweet_count|
+--------------------+-------------+
|La macronie En Ma...|            0|
|Choisir entre le ...|            0|
|@LeaFrct @Dormez_...|            0|
|@DELARUEEm @Miche...|            0|
|@MathiasPangeois ...|            0|
|@MarylinMaeso @En...|            0|
|@bruno_salomon @p...|            0|
|@MathiasPangeois ...|            0|
|#CGT #CFDT font b...|            0|
|#macron : «Interd...|            0|
|@stefappriou @Syl...|            0|
|@RabierJF @Emmanu...|            0|
|@AxelBenhy @joelc...|            0|
|Message de Macron...|            0|
|@MarieMarysienka ...|            0|
|@etienneliebig Ca...|            0|
|@MarieMarysienka ...|            0|
|#macron aux trava...|            0|
|Voilà absolument ...|            0|
|@GeorgesFENECH @j...|            0|
+--------------------+-------------+
only showing top 20 rows



In [5]:
tweets_df = spark.read.json('./data/french_tweets.json')
# Format dates from 'created_at' column
date_format_udf = udf(lambda datecol: datetime.strptime(datecol, '%a %b %d %X %z %Y'), DateType())
tweets_df = tweets_df.filter(col('created_at').isNotNull()).withColumn('creation_date', date_format_udf(col('created_at')))
tweets_count = tweets_df.count()
print('Total tweet count: {}'.format(tweets_count))

NameError: name 'col' is not defined